In [1]:
from __future__ import print_function

from keras.engine import Model
from keras_vggface.vggface import VGGFace
from keras.preprocessing import image
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers.core import Dense

import numpy as np

from pre_processing import load_data, split_train_test

Using TensorFlow backend.


In [20]:
# Load the data.
x, y, kmasks, vocab = load_data(k=8, speakers=[1], shuffle=True, use_delta_frames=True)

Skipped 5 empty data files.


In [3]:
vocab.occurrences()

{'a': 40,
 'again': 250,
 'at': 252,
 'b': 40,
 'bin': 240,
 'blue': 250,
 'by': 248,
 'c': 40,
 'd': 40,
 'e': 40,
 'eight': 100,
 'f': 40,
 'five': 100,
 'four': 100,
 'g': 40,
 'green': 250,
 'h': 40,
 'i': 40,
 'in': 252,
 'j': 40,
 'k': 40,
 'l': 40,
 'lay': 248,
 'm': 40,
 'n': 40,
 'nine': 100,
 'now': 250,
 'o': 40,
 'one': 100,
 'p': 40,
 'place': 256,
 'please': 250,
 'q': 40,
 'r': 40,
 'red': 250,
 's': 40,
 'set': 256,
 'seven': 100,
 'six': 100,
 'soon': 250,
 't': 40,
 'three': 100,
 'two': 100,
 'u': 40,
 'v': 40,
 'white': 250,
 'with': 248,
 'x': 40,
 'y': 40,
 'z': 40,
 'zero': 100}

In [21]:
# Select which fold of the k-folds to use.
FOLD = 0

train, test = split_train_test(x, y, kmasks[FOLD])
x_train, y_train = train
x_test, y_test = test

print('Number of words for training: ', x_train.shape[0])
print('Number of words for testing: ', x_test.shape[0])
print('Frames per word: ', x_train.shape[1])
print('Features per frame: ', x_train.shape[2])

Number of words for training:  5250
Number of words for testing:  750
Frames per word:  6
Features per frame:  512


In [22]:
# Build the LSTM model.
model = Sequential()
model.add(LSTM(128, input_shape=x_train[0].shape, return_sequences=True))
model.add(LSTM(128))
model.add(Dense(len(vocab), activation='tanh'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['accuracy'])

In [23]:
# Train the model.
model.fit(x_train, y_train, batch_size=32, epochs=20, verbose=1)

Epoch 1/20
5250/5250 [==============================] - 4s - loss: 0.0170 - acc: 0.3541          

In [24]:
# Test the model.
score, acc = model.evaluate(x_test, y_test, batch_size=32)
print('Test score: ', score)
print('Test accuracy: ', acc)

672/750 [=========================>....] - ETA: 0sTest score:  0.00572603820885
Test accuracy:  0.821333333492


In [19]:
# Try matching some results.
frames_per_word = x_test.shape[1]
features_per_frame = x_test.shape[2]

RESULTS_TO_CHECK = 50
MATCHES_PER_WORD = 4

for i in xrange(RESULTS_TO_CHECK):
    pred = model.predict(x_test[i].reshape(1, frames_per_word, features_per_frame))
    predicted_indexes = np.argsort(pred.reshape(len(vocab)))[::-1][:MATCHES_PER_WORD]
    correct_index = np.argsort(y_test[i])[::-1][0]

    correct_word = vocab[correct_index]
    predicted_words = [vocab[i] for i in predicted_indexes]

    print('{} : {}'.format(correct_word, predicted_words))

place : ['blue', 't', 'd', 'in']
at : ['place', 'x', 'eight', 'n']
now : ['a', 'i', 'd', 'n']
lay : ['lay', 'place', 'set', 'h']
red : ['bin', 'b', 'please', 'a']
five : ['four', 'v', 's', 'p']
set : ['set', 'lay', 'place', 't']
one : ['with', 'white', 'n', 'seven']
two : ['two', 'nine', 'u', 'i']
place : ['place', 'h', 'p', 'k']
f : ['at', 'f', 'i', 'n']
lay : ['lay', 'z', 't', 'g']
again : ['d', 'blue', 'k', 't']
d : ['lay', 'zero', 'd', 'seven']
lay : ['lay', 'place', 'green', 't']
by : ['blue', 't', 'by', 'd']
in : ['with', 'one', 'y', 'nine']
seven : ['six', 'd', 'z', 't']
please : ['in', 'e', 'u', 'a']
bin : ['blue', 'four', 't', 'd']
set : ['g', 'lay', 't', 'by']
red : ['with', 'v', 'blue', 'seven']
please : ['please', 'p', 'b', 'e']
s : ['s', 'a', 'at', 'three']
seven : ['e', 'at', 'seven', 'd']
again : ['e', 'q', 'k', 'n']
in : ['with', 'one', 'y', 'red']
again : ['q', 'u', 'o', 'two']
set : ['at', 'lay', 'in', 't']
eight : ['u', 'in', 'q', 'r']
red : ['bin', 'blue', 'b', 'p']